In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import make_scorer, mean_squared_error

Neural Network

In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/CSCI567/CSCI567/writing_quality/final_df.csv')

X = df.drop(['id', 'score'], axis=1)
y = df['score']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Neural network model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(16, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(8, activation='relu'),
    layers.Flatten(),
    layers.Dense(1, activation='linear')
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, verbose=False)

y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error on Test Set: {rmse}')


16/16 [==============================] - 0s 2ms/step
Root Mean Squared Error on Test Set: 0.7824294685083744


MLP with Grid Search Cross Validation

In [ ]:
data = pd.read_csv('/content/drive/Shareddrives/CSCI567/CSCI567/writing_quality/final_df.csv')

label_encoder = LabelEncoder()
data['score'] = label_encoder.fit_transform(data['score'])

X = data.drop(['score', 'id'], axis=1)
y = data['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model defintiton
model = MLPClassifier(max_iter=1000, random_state=42, early_stopping=True, validation_fraction=0.1, n_iter_no_change=10, warm_start=True)

# grid search cv
param_grid = {
    'hidden_layer_sizes': [(50, 50, 25), (100, 50, 25), (200, 100, 50)],
    'solver': ['adam', 'sgd'],
    'activation': ['logistic', 'tanh', 'relu'],
    'learning_rate_init': [0.01, 0.001, 0.0001]
}

scorer = make_scorer(mean_squared_error, squared=False)

# CV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring=scorer, verbose=1, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

print("Best Hyperparameters:", grid_search.best_params_)

best_model = grid_search.best_estimator_

# Predictions
y_pred = best_model.predict(X_test_scaled)

# Inverse transform
y_pred_original = label_encoder.inverse_transform(y_pred)
y_test_original = label_encoder.inverse_transform(y_test)

# RMSE
rmse = np.sqrt(mean_squared_error(y_test_original, y_pred_original))
print(f'Root Mean Squared Error (RMSE) on Test Set: {rmse}')
